In [49]:
!python3 main.py play --agents q_agent --train 1 --scenario coin-heaven --n-rounds 10000 --no-gui

size of dictionary: 70
  0%|                                                    | 0/20 [00:00<?, ?it/s]
random
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.


random
check danger (3, 15)
area [[1, 15], [2, 15], [4, 15], [5, 15], [6, 15], [3, 12], [3, 13], [3, 14], [3, 15]]
bombs 1
field pos 0
bomb true ((1, 15), 0)
check danger (3, 15)
area [[1, 15], [2, 15], [4, 15], [5, 15], [6, 15], [3, 12], [3, 13], [3, 14], [3, 15]]
bombs 1
field pos 0
bomb true ((1, 15), 0)
check danger (3, 15)
area [[1, 15], [2, 15], [4, 15], [5, 15], [6, 15], [3, 12], [3, 13], [3, 14], [3, 15]]
bombs 1
field pos 0
bomb true ((1, 15), 0)
check danger [2, 15]
area [[1, 15], [3, 15], [4, 15], [5, 15], [2, 15]]
bombs 1
field pos 0
bomb true ((1, 15), 0)
check danger [4, 15]
area [[1, 15], [2, 15], [3, 15], [5, 15], [6, 15], [7, 15], [4, 15]]
bombs 1
field pos 0
bomb true ((1, 15), 0)
check danger [3, 14]
area [[3, 11], [3, 12], [3, 13], [3, 14]]
bombs 1
field pos 0
check danger [ 2 15]
area [[1, 15], [3, 15], [4, 15], [5, 15], [2, 15]]
bombs 1
field pos 0
bomb true ((1, 15), 0)
d, i 2.0 0
argmax 2
check danger [ 4 15]
area [[1, 15], [2, 15], [3, 15], [5, 15], [6, 15], [

check danger [2 1]
area [[1, 1], [3, 1], [4, 1], [5, 1], [2, 1]]
bombs 0
field pos 0
features [3, 0, -1, 2, 0, -1]
check danger (1, 1)
area [[2, 1], [3, 1], [4, 1], [1, 2], [1, 3], [1, 4], [1, 1]]
bombs 0
field pos 0
check danger (1, 1)
area [[2, 1], [3, 1], [4, 1], [1, 2], [1, 3], [1, 4], [1, 1]]
bombs 0
field pos 0
check danger [0 1]
area [[1, 1], [2, 1], [3, 1], [0, 1]]
bombs 0
field pos -1
check danger [1 2]
area [[1, 3], [1, 4], [1, 5], [1, 2]]
bombs 0
field pos 0
check danger [2 1]
area [[1, 1], [3, 1], [4, 1], [5, 1], [2, 1]]
bombs 0
field pos 0
features [3, 0, -1, 2, 0, -1]

random
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.


random
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
events ['MOVED_UP', 'COIN_DISTANCE_INCREASED']
check danger (1, 2)
area [

check danger [2 1]
area [[1, 1], [3, 1], [4, 1], [5, 1], [2, 1]]
bombs 0
field pos 0
features [3, 0, -1, 2, 0, -1]
check danger (1, 1)
area [[2, 1], [3, 1], [4, 1], [1, 2], [1, 3], [1, 4], [1, 1]]
bombs 0
field pos 0
check danger (1, 1)
area [[2, 1], [3, 1], [4, 1], [1, 2], [1, 3], [1, 4], [1, 1]]
bombs 0
field pos 0
check danger [0 1]
area [[1, 1], [2, 1], [3, 1], [0, 1]]
bombs 0
field pos -1
check danger [1 2]
area [[1, 3], [1, 4], [1, 5], [1, 2]]
bombs 0
field pos 0
check danger [2 1]
area [[1, 1], [3, 1], [4, 1], [5, 1], [2, 1]]
bombs 0
field pos 0
features [3, 0, -1, 2, 0, -1]

random
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.

...

check danger [6 1]
area [[3, 1], [4, 1], [5, 1], [7, 1], [8, 1], [9, 1], [6, 1]]
bombs 0
field pos 0
features [3, 0, 0, 0, 2, -1]
check danger (5, 2)
area [[5, 3], [5, 4], [5, 5], [5, 2]]
bombs 0
field pos 0
check danger (5, 2)
area [[5, 3], [5, 4], [5, 5], [5, 2]]
bombs 0
field pos 0
check danger [4 2]
area [[3, 2], [5, 2], [4, 5], [4, 2]]
bombs 0
field pos -1
check danger [5 3]
area [[2, 3], [3, 3], [4, 3], [6, 3], [7, 3], [8, 3], [5, 4], [5, 5], [5, 6], [5, 3]]
bombs 0
field pos 0
check danger [5 1]
area [[2, 1], [3, 1], [4, 1], [6, 1], [7, 1], [8, 1], [5, 2], [5, 3], [5, 4], [5, 1]]
bombs 0
field pos 0
features [3, 0, -1, 2, -1, 0]

random
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0

check danger [5 4]
area [[5, 1], [5, 2], [5, 3], [5, 5], [5, 6], [5, 7], [5, 4]]
bombs 0
field pos 0
check danger [6 3]
area [[3, 3], [4, 3], [5, 3], [7, 3], [8, 3], [9, 3], [6, 3]]
bombs 0
field pos 0
check danger [5 2]
area [[5, 3], [5, 4], [5, 5], [5, 2]]
bombs 0
field pos 0
features [3, 0, 0, 0, 2, 0]
check danger (5, 3)
area [[2, 3], [3, 3], [4, 3], [6, 3], [7, 3], [8, 3], [5, 4], [5, 5], [5, 6], [5, 3]]
bombs 1
field pos 0
bomb true ((5, 3), 3)
check danger (5, 3)
area [[2, 3], [3, 3], [4, 3], [6, 3], [7, 3], [8, 3], [5, 4], [5, 5], [5, 6], [5, 3]]
bombs 1
field pos 0
check danger [4 3]
area [[1, 3], [2, 3], [3, 3], [5, 3], [6, 3], [7, 3], [4, 3]]
bombs 1
field pos 0
check danger [4 3]
area [[1, 3], [2, 3], [3, 3], [5, 3], [6, 3], [7, 3], [4, 3]]
bombs 1
field pos 0
bomb true ((5, 3), 3)
d, i 0.0 0
argmax 0
check danger [5 4]
area [[5, 1], [5, 2], [5, 3], [5, 5], [5, 6], [5, 7], [5, 4]]
bombs 1
field pos 0
bomb true ((5, 3), 3)
d, i 0.0 0
argmax 0
check danger [6 3]
area [[3, 3],

check danger [8 3]
area [[5, 3], [6, 3], [7, 3], [9, 3], [10, 3], [11, 3], [8, 3]]
bombs 1
field pos 0
bomb true ((5, 3), 0)
d, i 2.0 0
argmax 2
check danger [7 2]
area [[7, 3], [7, 4], [7, 5], [7, 2]]
bombs 1
field pos 0
features [1, 0, 1, 0, 2, 0]

random
check danger (7, 3)
area [[4, 3], [5, 3], [6, 3], [8, 3], [9, 3], [10, 3], [7, 4], [7, 5], [7, 6], [7, 3]]
bombs 1
field pos 0
bomb true ((5, 3), 0)
check danger (7, 3)
area [[4, 3], [5, 3], [6, 3], [8, 3], [9, 3], [10, 3], [7, 4], [7, 5], [7, 6], [7, 3]]
bombs 1
field pos 0
bomb true ((5, 3), 0)
check danger (7, 3)
area [[4, 3], [5, 3], [6, 3], [8, 3], [9, 3], [10, 3], [7, 4], [7, 5], [7, 6], [7, 3]]
bombs 1
field pos 0
bomb true ((5, 3), 0)
check danger [6, 3]
area [[3, 3], [4, 3], [5, 3], [7, 3], [8, 3], [9, 3], [6, 3]]
bombs 1
field pos 0
bomb true ((5, 3), 0)
check danger [7, 4]
area [[7, 1], [7, 2], [7, 3], [7, 5], [7, 6], [7, 7], [7, 4]]
bombs 1
field pos 0
check danger [8, 3]
area [[5, 3], [6, 3], [7, 3], [9, 3], [10, 3], [1

features [3, 0, 2, -1, -1, 0]

random
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
events ['MOVED_LEFT', 'COIN_COLLECTED', 'CO

check danger [14 15]
area [[11, 15], [12, 15], [13, 15], [15, 15], [14, 15]]
bombs 0
field pos 0
check danger [13 14]
area [[13, 11], [13, 12], [13, 13], [13, 14]]
bombs 0
field pos 0
features [3, 0, 2, -1, 0, 0]
check danger (13, 14)
area [[13, 11], [13, 12], [13, 13], [13, 14]]
bombs 0
field pos 0
check danger (13, 14)
area [[13, 11], [13, 12], [13, 13], [13, 14]]
bombs 0
field pos 0
check danger [12 14]
area [[11, 14], [13, 14], [12, 11], [12, 14]]
bombs 0
field pos -1
check danger [13 15]
area [[10, 15], [11, 15], [12, 15], [14, 15], [15, 15], [13, 12], [13, 13], [13, 14], [13, 15]]
bombs 0
field pos 0
check danger [13 13]
area [[10, 13], [11, 13], [12, 13], [14, 13], [15, 13], [13, 10], [13, 11], [13, 12], [13, 13]]
bombs 0
field pos 0
features [3, 0, -1, 2, -1, 0]

random
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [29]:
!python3 main.py play --agents q_agent --scenario coin-heaven --n-rounds 5

size of dictionary: 70
nonzero rows: 5
  0%|                                                     | 0/5 [00:00<?, ?it/s]my_pos  (1, 15)
neigh  [array([ 0, 15]), array([ 1, 16]), array([ 2, 15]), array([ 1, 14])]
n_values 1  [-1, -1, 0, 0]
check danger
check danger
check danger
check danger
coins, j 1 2
coins, j 1 2
2 0 True 2
check danger
coins, j 1 2
coins, j 1 2
3 0 True 2
features [3, 0, -1, -1, 2, 0]
state, i, r, a [ 3  0 -1 -1  2  0] 215 0 1
RIGHT
q-table [-0.6  0.   0.   0.   0.   0. ]
my_pos  (2, 15)
neigh  [array([ 1, 15]), array([ 2, 16]), array([ 3, 15]), array([ 2, 14])]
n_values 1  [0, -1, 0, -1]
check danger
check danger
check danger
check danger
coins, j 1 0
coins, j 1 0
0 0 True 0
check danger
coins, j 1 0
coins, j 1 0
2 0 True 0
features [3, 0, 2, -1, 0, -1]
state, i, r, a [ 3  0  2 -1  0 -1] 315 1 0
new action 3
LEFT
q-table [0. 0. 0. 0. 0. 0.]
my_pos  (1, 15)
neigh  [array([ 0, 15]), array([ 1, 16]), array([ 2, 15]), array([ 1, 14])]
n_values 1  [-1, -1, 0, 0]
check da

my_pos  (3, 15)
neigh  [array([ 2, 15]), array([ 3, 16]), array([ 4, 15]), array([ 3, 14])]
n_values 1  [0, -1, 0, 0]
check danger
check danger
check danger
check danger
coins, j 0 0
coins, j 1 2
coins, j 1 2
0 0 True 2
check danger
coins, j 0 0
coins, j 1 2
coins, j 1 2
2 0 True 2
check danger
coins, j 0 0
coins, j 1 2
coins, j 1 2
3 0 True 2
features [3, 0, 0, -1, 2, 0]
state, i, r, a [ 3  0  0 -1  2  0] 755 1 0
new action 3
LEFT
q-table [0. 0. 0. 0. 0. 0.]
my_pos  (2, 15)
neigh  [array([ 1, 15]), array([ 2, 16]), array([ 3, 15]), array([ 2, 14])]
n_values 1  [0, -1, 0, -1]
check danger
check danger
check danger
check danger
coins, j 0 0
coins, j 1 2
0 0 True 2
check danger
coins, j 0 0
coins, j 1 2
2 0 True 2
features [3, 0, 0, -1, 2, -1]
state, i, r, a [ 3  0  0 -1  2 -1] 315 3 0
new action 1
RIGHT
q-table [0. 0. 0. 0. 0. 0.]
my_pos  (3, 15)
neigh  [array([ 2, 15]), array([ 3, 16]), array([ 4, 15]), array([ 3, 14])]
n_values 1  [0, -1, 0, 0]
check danger
check danger
check danger
c

In [38]:
#--my-agents --save-replay --turn-based --save-stats

In [ ]:
!python main.py replay "replays/Round 07 (2022-03-23 12-52-13).pt" --turn-based

In [ ]:
reward = np.load("agent_code/q_agent/rewards.npy")
plt.plot(reward,'.')
plt.xlabel("rounds")
plt.ylabel("reward")
#plt.ylim(-100,100)

In [ ]:
model_trained = np.load("agent_code/q_agent/my-saved-model.npy")

In [19]:
import numpy as np
import random


In [6]:
def dist(pos, objects):
    return np.sqrt( np.power(np.subtract(objects, pos).transpose()[0], 2) + np.power(np.subtract(objects, pos).transpose()[1], 2) )


pos = (4,1)
ob = [(1,1)]
print(np.subtract(ob, pos))
print(dist(pos,ob))

a = (1,1)
b = [1,1]
print((np.array(a)==b).all())

def get_down(pos, n=3):
        down = []
        for x in range(pos[1]+1, pos[1]+(n+1)):
            if (0 <= x < 17):
                down.append([pos[0], x])
        return down
    
print(get_down((1,1)))

[[-3  0]]
[3.]
True
[[1, 2], [1, 3], [1, 4]]


In [7]:
def get_neighbours(pos):
    sub = [(1,0), (0,-1), (-1,0), (0,1)] # left, down, right, up

    neighbours = []
    for i in sub:
        neighbour = np.subtract(pos, i)
        if (0 <= neighbour[0] < 17) and (0 <= neighbour[1] < 17): # check if in game borders
            neighbours.append(neighbour)
    return neighbours, pos

print(get_neighbours((1,15)))

[array([ 0, 15]), array([ 1, 16]), array([ 2, 15]), array([ 1, 14])]


In [48]:
pos = np.array((1,2))
area = [[1, 3], [1, 4], [1, 5], [1, 2]]
for a in area:
    print((a == np.array(pos)).all())

False
False
False
True


In [68]:
from datetime import datetime

now = datetime.now() # current date and time

year = now.strftime("%Y")
print("year:", year)

month = now.strftime("%m")
print("month:", month)

day = now.strftime("%d")
print("day:", day)

time = now.strftime("%H:%M:%S")
print("time:", time)

date_time = now.strftime("%H_%M_%S")
print("date and time:",date_time)

year: 2022
month: 03
day: 23
time: 18:59:01
date and time: 18_59_01


In [ ]:
import time
start = time.time()

time.sleep(5)
end  = time.time()
print(end-start)